In [87]:
import pandas as pd
import numpy as np
import lux
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.decomposition import TruncatedSVD, NMF

In [49]:
import requests 

docs_url = 'https://raw.githubusercontent.com/alexeygrigorev/llm-rag-workshop/main/notebooks/documents.json'
docs_response = requests.get(docs_url)
documents_raw = docs_response.json()

documents = []

for course in documents_raw:
    course_name = course['course']

    for doc in course['documents']:
        doc['course'] = course_name
        documents.append(doc)

In [50]:
df = pd.DataFrame(documents, columns=['course', 'section', 'question', 'text'])
df.head()

/Users/damien/Documents/Dev/LLM_Zoomcamp/search_engine_implementation/.venv/lib/python3.13/site-packages/lux/executor/PandasExecutor.py:548: UserWarning:Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
/Users/damien/Documents/Dev/LLM_Zoomcamp/search_engine_implementation/.venv/lib/python3.13/site-packages/lux/executor/PandasExecutor.py:548: UserWarning:Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
/Users/damien/Documents/Dev/LLM_Zoomcamp/search_engine_implementation/.venv/lib/python3.13/site-packages/lux/executor/PandasExecutor.py:548: UserWarning:Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
/Users/damien/Documents/Dev/

Button(description='Toggle Pandas/Lux', layout=Layout(top='5px', width='140px'), style=ButtonStyle())

Output()

In [51]:
cv = CountVectorizer()
cv.fit(df.text)

,"input input: {'filename', 'file', 'content'}, default='content'- If `'filename'`, the sequence passed as an argument to fit is expected to be a list of filenames that need reading to fetch the raw content to analyze.- If `'file'`, the sequence items must have a 'read' method (file-like object) that is called to fetch the bytes in memory.- If `'content'`, the input is expected to be a sequence of items that can be of type string or byte.",'content'
,"encoding encoding: str, default='utf-8'If bytes or files are given to analyze, this encoding is used todecode.",'utf-8'
,"decode_error decode_error: {'strict', 'ignore', 'replace'}, default='strict'Instruction on what to do if a byte sequence is given to analyze thatcontains characters not of the given `encoding`. By default, it is'strict', meaning that a UnicodeDecodeError will be raised. Othervalues are 'ignore' and 'replace'.",'strict'
,"strip_accents strip_accents: {'ascii', 'unicode'} or callable, default=NoneRemove accents and perform other character normalizationduring the preprocessing step.'ascii' is a fast method that only works on characters that havea direct ASCII mapping.'unicode' is a slightly slower method that works on any characters.None (default) means no character normalization is performed.Both 'ascii' and 'unicode' use NFKD normalization from:func:`unicodedata.normalize`.",None
,"lowercase lowercase: bool, default=TrueConvert all characters to lowercase before tokenizing.",True
,"preprocessor preprocessor: callable, default=NoneOverride the preprocessing (strip_accents and lowercase) stage whilepreserving the tokenizing and n-grams generation steps.Only applies if ``analyzer`` is not callable.",None
,"tokenizer tokenizer: callable, default=NoneOverride the string tokenization step while preserving thepreprocessing and n-grams generation steps.Only applies if ``analyzer == 'word'``.",None
,"stop_words stop_words: {'english'}, list, default=NoneIf 'english', a built-in stop word list for English is used.There are several known issues with 'english' and you shouldconsider an alternative (see :ref:`stop_words`).If a list, that list is assumed to contain stop words, all of whichwill be removed from the resulting tokens.Only applies if ``analyzer == 'word'``.If None, no stop words will be used. In this case, setting `max_df`to a higher value, such as in the range (0.7, 1.0), can automatically detectand filter stop words based on intra corpus document frequency of terms.",None
,"token_pattern token_pattern: str or None, default=r""(?u)\\b\\w\\w+\\b""Regular expression denoting what constitutes a ""token"", only usedif ``analyzer == 'word'``. The default regexp select tokens of 2or more alphanumeric characters (punctuation is completely ignoredand always treated as a token separator).If there is a capturing group in token_pattern then thecaptured group content, not the entire match, becomes the token.At most one capturing group is permitted.",'(?u)\\b\\w\\w+\\b'
,"ngram_range ngram_range: tuple (min_n, max_n), default=(1, 1)The lower and upper boundary of the range of n-values for differentword n-grams or char n-grams to be extracted. All values of n suchsuch that min_n <= n <= max_n will be used. For example an``ngram_range`` of ``(1, 1)`` means only unigrams, ``(1, 2)`` meansunigrams and bigrams, and ``(2, 2)`` means only bigrams.Only applies if ``analyzer`` is not callable.","(1, ...)"
,"analyzer analyzer: {'word', 'char', 'char_wb'} or callable, default='word'Whether the feature should be made of word n-gram or charactern-grams.Option 'char_wb' creates character n-grams only from text insideword boundaries; n-grams at the edges of words are padded with space.If a callable is passed it is used to extract the sequence of featuresout of the raw, unprocessed input... versionchanged:: 0.21Since v0.21, if ``input`` is ``filename`` or ``file``, the data isfirst read from the file and then passed to the given callableanalyzer.",'word'


In [52]:
cv.get_feature_names_out().shape

(6711,)

Create a full Matrix with the 6711 columns, but we don't care about all columns

In [53]:
cv_opti = CountVectorizer(min_df=5)
cv_opti.fit(df.text)

,"input input: {'filename', 'file', 'content'}, default='content'- If `'filename'`, the sequence passed as an argument to fit is expected to be a list of filenames that need reading to fetch the raw content to analyze.- If `'file'`, the sequence items must have a 'read' method (file-like object) that is called to fetch the bytes in memory.- If `'content'`, the input is expected to be a sequence of items that can be of type string or byte.",'content'
,"encoding encoding: str, default='utf-8'If bytes or files are given to analyze, this encoding is used todecode.",'utf-8'
,"decode_error decode_error: {'strict', 'ignore', 'replace'}, default='strict'Instruction on what to do if a byte sequence is given to analyze thatcontains characters not of the given `encoding`. By default, it is'strict', meaning that a UnicodeDecodeError will be raised. Othervalues are 'ignore' and 'replace'.",'strict'
,"strip_accents strip_accents: {'ascii', 'unicode'} or callable, default=NoneRemove accents and perform other character normalizationduring the preprocessing step.'ascii' is a fast method that only works on characters that havea direct ASCII mapping.'unicode' is a slightly slower method that works on any characters.None (default) means no character normalization is performed.Both 'ascii' and 'unicode' use NFKD normalization from:func:`unicodedata.normalize`.",None
,"lowercase lowercase: bool, default=TrueConvert all characters to lowercase before tokenizing.",True
,"preprocessor preprocessor: callable, default=NoneOverride the preprocessing (strip_accents and lowercase) stage whilepreserving the tokenizing and n-grams generation steps.Only applies if ``analyzer`` is not callable.",None
,"tokenizer tokenizer: callable, default=NoneOverride the string tokenization step while preserving thepreprocessing and n-grams generation steps.Only applies if ``analyzer == 'word'``.",None
,"stop_words stop_words: {'english'}, list, default=NoneIf 'english', a built-in stop word list for English is used.There are several known issues with 'english' and you shouldconsider an alternative (see :ref:`stop_words`).If a list, that list is assumed to contain stop words, all of whichwill be removed from the resulting tokens.Only applies if ``analyzer == 'word'``.If None, no stop words will be used. In this case, setting `max_df`to a higher value, such as in the range (0.7, 1.0), can automatically detectand filter stop words based on intra corpus document frequency of terms.",None
,"token_pattern token_pattern: str or None, default=r""(?u)\\b\\w\\w+\\b""Regular expression denoting what constitutes a ""token"", only usedif ``analyzer == 'word'``. The default regexp select tokens of 2or more alphanumeric characters (punctuation is completely ignoredand always treated as a token separator).If there is a capturing group in token_pattern then thecaptured group content, not the entire match, becomes the token.At most one capturing group is permitted.",'(?u)\\b\\w\\w+\\b'
,"ngram_range ngram_range: tuple (min_n, max_n), default=(1, 1)The lower and upper boundary of the range of n-values for differentword n-grams or char n-grams to be extracted. All values of n suchsuch that min_n <= n <= max_n will be used. For example an``ngram_range`` of ``(1, 1)`` means only unigrams, ``(1, 2)`` meansunigrams and bigrams, and ``(2, 2)`` means only bigrams.Only applies if ``analyzer`` is not callable.","(1, ...)"
,"analyzer analyzer: {'word', 'char', 'char_wb'} or callable, default='word'Whether the feature should be made of word n-gram or charactern-grams.Option 'char_wb' creates character n-grams only from text insideword boundaries; n-grams at the edges of words are padded with space.If a callable is passed it is used to extract the sequence of featuresout of the raw, unprocessed input... versionchanged:: 0.21Since v0.21, if ``input`` is ``filename`` or ``file``, the data isfirst read from the file and then passed to the given callableanalyzer.",'word'


In [54]:
cv_opti.get_feature_names_out()

array(['01', '02', '03', ..., 'youtube', 'zip', 'zoomcamp'],
      shape=(1524,), dtype=object)

In [55]:
docs_example = [
    "Course starts on 15th Jan 2024, 15th Jan 2024 google",
    "Prerequisites listed on GitHub google",
    "Submit homeworks after start date google",
    "Registration not required for participation google",
    "Setup Google Cloud and Python before course google"
]
cv_example = CountVectorizer(stop_words="english")
cv_example.fit(docs_example)
cv_example.get_feature_names_out()

array(['15th', '2024', 'cloud', 'course', 'date', 'github', 'google',
       'homeworks', 'jan', 'listed', 'participation', 'prerequisites',
       'python', 'registration', 'required', 'setup', 'start', 'starts',
       'submit'], dtype=object)

In [56]:
X = cv_example.transform(docs_example)
pd.DataFrame(X.todense(), columns=cv_example.get_feature_names_out()).T

/Users/damien/Documents/Dev/LLM_Zoomcamp/search_engine_implementation/.venv/lib/python3.13/site-packages/lux/executor/PandasExecutor.py:567: UserWarning:Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
/Users/damien/Documents/Dev/LLM_Zoomcamp/search_engine_implementation/.venv/lib/python3.13/site-packages/lux/executor/PandasExecutor.py:567: UserWarning:Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
/Users/damien/Documents/Dev/LLM_Zoomcamp/search_engine_implementation/.venv/lib/python3.13/site-packages/lux/executor/PandasExecutor.py:567: UserWarning:Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
/Users/damien/Documents/Dev/

Button(description='Toggle Pandas/Lux', layout=Layout(top='5px', width='140px'), style=ButtonStyle())

Output()

In [57]:
tfif_c = TfidfVectorizer(stop_words='english')
tfif_c.fit(docs_example)
x_tfid = tfif_c.transform(docs_example)
pd.DataFrame(x_tfid.todense().round(2), columns=tfif_c.get_feature_names_out()).T

Button(description='Toggle Pandas/Lux', layout=Layout(top='5px', width='140px'), style=ButtonStyle())

Output()

In [58]:
cv_score = TfidfVectorizer(stop_words='english', min_df=5)
X = cv_score.fit_transform(df.text)

names = cv_score.get_feature_names_out()

In [59]:
query = "Do I need to know python to sign up for the January course?"

q = cv_score.transform([query])
q_array = q.toarray()
q_array

array([[0., 0., 0., ..., 0., 0., 0.]], shape=(1, 1333))

In [60]:

def filter_O(items):
    key, value = items
    if value > 0:
        return True
    else:
        return False

query_dict_filtered = dict(filter(filter_O,dict(zip(names, q.toarray()[0])).items()))
query_dict_filtered


{'course': np.float64(0.38148200594064524),
 'know': np.float64(0.5608269127690405),
 'need': np.float64(0.29796783250107517),
 'python': np.float64(0.31441356049301333),
 'sign': np.float64(0.5935519664108326)}

In [61]:

doc_dict_filtered = dict(filter(filter_O,dict(zip(names, X.toarray()[4])).items()))
doc_dict_filtered

{'account': np.float64(0.20994480766946202),
 'anaconda': np.float64(0.2518270791901051),
 'cloud': np.float64(0.3791044572650518),
 'dependencies': np.float64(0.275391199629744),
 'git': np.float64(0.21365330143755387),
 'google': np.float64(0.3684593731602147),
 'installed': np.float64(0.19922289760419193),
 'installing': np.float64(0.22672378916038158),
 'look': np.float64(0.24919683689936373),
 'python': np.float64(0.15689258057390296),
 'requirements': np.float64(0.2575305609565735),
 'sdk': np.float64(0.29014309970334246),
 'setting': np.float64(0.23194305125372439),
 'start': np.float64(0.20761356612294093),
 'terraform': np.float64(0.2575305609565735)}

Now that we have 2 vectors (the one from the query and the one from the doc), let's find out their similarity with dot product
And we can do it with all doc.

In [62]:
X.dot(q_array.T)

array([[0.19464486],
       [0.        ],
       [0.        ],
       [0.06011641],
       [0.04932915],
       [0.        ],
       [0.        ],
       [0.13477565],
       [0.        ],
       [0.        ],
       [0.        ],
       [0.15899187],
       [0.        ],
       [0.        ],
       [0.        ],
       [0.07431408],
       [0.        ],
       [0.        ],
       [0.05779673],
       [0.07243428],
       [0.        ],
       [0.05174293],
       [0.16373635],
       [0.08076031],
       [0.        ],
       [0.09755254],
       [0.        ],
       [0.21069625],
       [0.12067781],
       [0.        ],
       [0.        ],
       [0.        ],
       [0.        ],
       [0.06381749],
       [0.        ],
       [0.        ],
       [0.        ],
       [0.        ],
       [0.        ],
       [0.00910541],
       [0.02835681],
       [0.05480112],
       [0.        ],
       [0.        ],
       [0.        ],
       [0.        ],
       [0.        ],
       [0.024

In [63]:
cosine_similarity(X,q_array).round(4) == X.dot(q_array.T).round(4)

array([[ True],
       [ True],
       [ True],
       [ True],
       [ True],
       [ True],
       [ True],
       [ True],
       [ True],
       [ True],
       [ True],
       [ True],
       [ True],
       [ True],
       [ True],
       [ True],
       [ True],
       [ True],
       [ True],
       [ True],
       [ True],
       [ True],
       [ True],
       [ True],
       [ True],
       [ True],
       [ True],
       [ True],
       [ True],
       [ True],
       [ True],
       [ True],
       [ True],
       [ True],
       [ True],
       [ True],
       [ True],
       [ True],
       [ True],
       [ True],
       [ True],
       [ True],
       [ True],
       [ True],
       [ True],
       [ True],
       [ True],
       [ True],
       [ True],
       [ True],
       [ True],
       [ True],
       [ True],
       [ True],
       [ True],
       [ True],
       [ True],
       [ True],
       [ True],
       [ True],
       [ True],
       [ True],
       [

In [64]:
scores = cosine_similarity(X,q_array).flatten()
indexes_doc_relevant = np.argsort(scores) # gives the indexes of documents asc
indexes_doc_relevant[-5:]

array([764,  27, 806, 577, 445])

In [65]:
df.iloc[445].text

'Check this article. If you know everything in this article, you know enough. If you don’t, read the article and join the coursIntroduction to Pythone too :)\nIntroduction to Python – Machine Learning Bookcamp\nYou can follow this English course from the OpenClassrooms e-learning platform, which is free and covers the python basics for data analysis: Learn Python Basics for Data Analysis - OpenClassrooms . It is important to know some basics such as: how to run a Jupyter notebook, how to import libraries (and what libraries are), how to declare a variable (and what variables are) and some important operations regarding data analysis.\n(Mélanie Fouesnard)'

In [66]:
fields = ['section', 'question', 'text']
matrices = {}
vectorizers = {}
for f in fields:
    cv = TfidfVectorizer(stop_words='english', min_df=5)
    X = cv.fit_transform(df[f])
    matrices[f] = X
    vectorizers[f] = cv


In [67]:
matrices

{'section': <Compressed Sparse Row sparse matrix of dtype 'float64'
 	with 3090 stored elements and shape (948, 66)>,
 'question': <Compressed Sparse Row sparse matrix of dtype 'float64'
 	with 3431 stored elements and shape (948, 291)>,
 'text': <Compressed Sparse Row sparse matrix of dtype 'float64'
 	with 23808 stored elements and shape (948, 1333)>}

In [68]:
vectorizers

{'section': TfidfVectorizer(min_df=5, stop_words='english'),
 'question': TfidfVectorizer(min_df=5, stop_words='english'),
 'text': TfidfVectorizer(min_df=5, stop_words='english')}

In [69]:
score = np.zeros(len(df))

query = "Do I need to know python to sign up for the January course?"

boosts = {
    'question' : 3,
    'text' : 0.5
}
for f in fields:
    q = vectorizers[f].transform([query])
    X = matrices[f]

    f_score = cosine_similarity(X,q).flatten()

    boost = boosts.get(f, 1.0)
    score = score + boost * f_score

In [70]:
filters = {
    'course': 'data-engineering-zoomcamp'
}

In [71]:
for field, value in filters.items():
    mask = (df[field] == value).astype(int)
    score = score * mask


In [72]:
score

Button(description='Toggle Pandas/Lux', layout=Layout(top='5px', width='140px'), style=ButtonStyle())

Output()

In [73]:
idx = np.argsort(score).tail()
df.iloc[idx]

/Users/damien/Documents/Dev/LLM_Zoomcamp/search_engine_implementation/.venv/lib/python3.13/site-packages/lux/executor/PandasExecutor.py:548: UserWarning:Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
/Users/damien/Documents/Dev/LLM_Zoomcamp/search_engine_implementation/.venv/lib/python3.13/site-packages/lux/executor/PandasExecutor.py:548: UserWarning:Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
/Users/damien/Documents/Dev/LLM_Zoomcamp/search_engine_implementation/.venv/lib/python3.13/site-packages/lux/executor/PandasExecutor.py:548: UserWarning:Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
/Users/damien/Documents/Dev/

Button(description='Toggle Pandas/Lux', layout=Layout(top='5px', width='140px'), style=ButtonStyle())

Output()

# TruncatedSVD

In [83]:
X = matrices['text']
cv = vectorizers['text']
X


<Compressed Sparse Row sparse matrix of dtype 'float64'
	with 23808 stored elements and shape (948, 1333)>

In [ ]:
svd = TruncatedSVD(n_components=16)
X_emb = svd.fit_transform(X)

In [77]:
X.shape

(948, 1333)

In [76]:
X_emb.shape

(948, 16)

In [84]:
Q = cv.transform([query])
Q.shape


(1, 1333)

In [85]:
Q_emb = svd.transform(Q)
Q_emb.shape

(1, 16)

In [81]:
score = cosine_similarity(X_emb, Q_emb).flatten()
idx = np.argsort(-score)[:10]
list(df.loc[idx].text)

['Technically, yes. Advisable? Not really. Reasons:\nSome homework(s) asks for specific python library versions.\nAnswers may not match in MCQ options if using different languages other than Python 3.10 (the recommended version for 2023 cohort)\nAnd as for midterms/capstones, your peer-reviewers may not know these other languages. Do you want to be penalized for others not knowing these other languages?\nYou can create a separate repo using course’s lessons but written in other languages for your own learnings, but not advisable for submissions.\ntx[source]',
 'We won’t re-record the course videos. The focus of the course and the skills we want to teach remained the same, and the videos are still up-to-date.\nIf you haven’t taken part in the previous iteration, you can start watching the videos. It’ll be useful for you and you will learn new things. However, we recommend using Python 3.10 now instead of Python 3.8.',
 'Yes, if you finish at least 2 out of 3 projects and review 3 peers’

In [94]:
nmf = NMF(n_components=16)
W = nmf.fit_transform(X)
Q_emb = nmf.transform(Q)
score = cosine_similarity(W, Q_emb).flatten()
idx = np.argsort(-score)[:10]
df.loc[idx]
H = nmf.components_
feature_names = cv.get_feature_names_out()
H.shape

(16, 1333)

In [98]:
def print_topic(H, feature_names, topic_idx, top_n=10):
    topic = H[topic_idx]
    top_indices = np.argsort(topic)[::-1][:top_n]
    for i in top_indices:
        print(f"{feature_names[i]:<20} {topic[i]:.3f}")

print_topic(H, feature_names, topic_idx=3, top_n=10)

model                1.482
features             0.432
training             0.343
train                0.313
validation           0.256
fit                  0.236
feature              0.208
regression           0.197
keras                0.189
values               0.186
